In [1]:
import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
fn = 'drive/My Drive/dataset_text.txt'


In [4]:
import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


class DatasetSeq2Seq(Dataset):
    def __init__(self, file_name, train_lang='en', bos: str = '~', eos: str = '#'):

        with open(file_name, 'r') as f:
            train = f.readlines()

        self.input_sequnces_vocab = {'pad': 0, bos: 1, eos: 2}
        self.output_sequnces_vocab = {'pad': 0, bos: 1, eos: 2}

        self.input_sequnces = []
        self.output_sequnces = []

        n_input = 3
        n_output = 3
        for line in train:
            split_line = line.split('\t')

            sequence = [self.input_sequnces_vocab[bos]]

            for char in split_line[0].strip():
                if self.input_sequnces_vocab.get(char) is None:
                    self.input_sequnces_vocab[char] = n_input
                    n_input += 1
                sequence.append(self.input_sequnces_vocab[char])
            sequence.append(self.input_sequnces_vocab[eos])

            target = [self.output_sequnces_vocab[bos]]
            for char in split_line[2].strip():
                if self.output_sequnces_vocab.get(char) is None:
                    self.output_sequnces_vocab[char] = n_output
                    n_output += 1
                target.append(self.output_sequnces_vocab[char])
            target.append(self.output_sequnces_vocab[eos])

            self.input_sequnces.append(sequence)
            self.output_sequnces.append(target)

        self.target_decode = [k for k in self.output_sequnces_vocab.keys()]

    def __len__(self):
        return len(self.input_sequnces)

    def __getitem__(self, index):
        return {
            'data': self.input_sequnces[index],
            'target': self.output_sequnces[index],
        }


def collate_fn(input_data):
    data = []
    targets = []

    for item in input_data:
        data.append(torch.as_tensor(item['data']))
        targets.append(torch.as_tensor(item['target']))

    data = pad_sequence(data, batch_first=True, padding_value=0)
    targets = pad_sequence(targets, batch_first=True, padding_value=0)
    data_mask = data > 0
    targets_mask = targets > 0

    return {'data': data, 
            'target': targets, 
            'data_mask': data_mask, 
            'targets_mask': targets_mask,
            }

In [5]:
dataset = DatasetSeq2Seq(fn)

In [6]:
#padding
# seq1 = [1, 2, 3, 4]
# seq2 = [9, 7, 6, 4, 3, 7, 5]
# pad seq1 equal seq2
# seq1 = [1, 2, 3, 4, 0, 0, 0]
# concat(seq1, seq2) [[1, 2, 3, 4, 0, 0, 0],
#                     [9, 7, 6, 4, 3, 7, 5]]

In [7]:
import torch
import torch.nn as nn
import numpy as np


class ScaledDotProductAttention(nn.Module):
    ''' Scaled Dot-Product Attention '''

    def __init__(self, temperature=1, attn_dropout=0.1):
        super().__init__()
        self.temperature = temperature
        self.dropout = nn.Dropout(attn_dropout)
        self.softmax = nn.Softmax(dim=2)

    def forward(self, q, k, v, mask=None):

        attn = torch.bmm(q, # B x T1 x V
                         k.transpose(1, 2), # B x T2 x V -> B x V x T2
                         ) # B x T1 x T2
        attn = attn / self.temperature

        if mask is not None:
            attn = attn.masked_fill(~mask, -np.inf)

        attn = self.softmax(attn)

        if mask is not None:
            attn = attn.masked_fill(~mask, 0.)

        attn = self.dropout(attn)
        output = torch.bmm(attn, v) # B x T1 x T2 @ B x T1 x V

        return output, attn

In [26]:
class Encoder(nn.Module):
    def __init__(self, vocab_len, emb_dim, hidden_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab_len, emb_dim)
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True)

    def forward(self, x):
        emb = self.emb(x)
        encoded_sequence, context = self.rnn(emb)

        return context, encoded_sequence

class Decoder(nn.Module):
    def __init__(self, vocab_len, emb_dim, hidden_dim, eos_id):
        super().__init__()
        self.emb = nn.Embedding(vocab_len, emb_dim)
        self.rnn = nn.GRU(emb_dim, hidden_dim, batch_first=True)
        self.classifier = nn.Linear(2*hidden_dim, vocab_len)
        self.do = nn.Dropout(0.2)
        self.eos = eos_id
        self.attention = ScaledDotProductAttention()

    def forward(self, context, target_sequence, encoded_sequence):
        if self.training:
            emb = self.emb(target_sequence)
            all_hid, _ = self.rnn(emb, context)
            attn_res, attn_mtx = self.attention(all_hid, 
                                                encoded_sequence, 
                                                encoded_sequence)
            clf_inp = torch.cat((attn_res, all_hid), dim=-1)
            pred_cls = self.classifier(self.do(clf_inp))

            return pred_cls
        else:
            predicts = []
            probas = []
            predicted_token = target_sequence #on this step target sequence contains bos only
            i = 0
            while predicted_token.item() != self.eos and i < 20:
                emb = self.emb(predicted_token)
                context, _ = self.rnn(emb, context) # context B x 1 x Hid
                attn_res, attn_mtx = self.attention(context, 
                                                    encoded_sequence, 
                                                    encoded_sequence)
                clf_inp = torch.cat((attn_res, context), dim=-1)
                pred = self.classifier(self.do(clf_inp))
                predicted_token = torch.argmax(pred, dim=-1) # B x 1
                predicts.append(predicted_token)
                probas.append(torch.softmax(pred, dim=-1))
                i += 1
            probas = torch.cat(probas, dim=1)
            
            return torch.cat(predicts, dim=1)

In [27]:
class DateNormalizer(nn.Module):
    def __init__(self, input_vocab_len, target_vocab_len, 
                 emb__dim, hidden_dim, eos_id):
        super().__init__()
        self.encoder = Encoder(input_vocab_len, emb_dim, hidden_dim)
        self.decoder = Decoder(target_vocab_len, emb_dim, hidden_dim, eos_id)

    def forward(self, x, target_sequence):
        context, encoded_sequece = self.encoder(x)
        pred = self.decoder(context, target_sequence, encoded_sequece)

        return pred

In [28]:
#hyper params
input_vocab_size = len(dataset.input_sequnces_vocab)
target_vocab_size = len(dataset.output_sequnces_vocab)
eos_id = dataset.output_sequnces_vocab['#']
#TODO try to use other model parameters
emb_dim = 128
hidden = 256
n_epochs = 20
batch_size = 128
cuda_device = -1
batch_size = 100
device = f'cuda:{cuda_device}' if cuda_device != -1 else 'cpu'

In [29]:
model = DateNormalizer(input_vocab_size, target_vocab_size, emb_dim, hidden, eos_id)
model.train()
optim = torch.optim.Adam(model.parameters(), lr=0.001)
loss_func = nn.CrossEntropyLoss()

In [30]:

for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()
        target = batch['target'].to(device)
        predicts = model(batch['data'].to(device), target[:, :-1])
        loss = loss_func(predicts.reshape(-1, target_vocab_size), 
                         target[:, 1:].reshape(-1))
        
        loss.backward()
        optim.step()
        if i % 10 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
    test = '14 мая 1978'
    test_encoded = torch.tensor([[dataset.input_sequnces_vocab[c] for c in test]])
    test_encoded = test_encoded.to(device)
    bos_input = torch.tensor([[dataset.output_sequnces_vocab['~']]]).to(device)
    with torch.no_grad():
        model.eval()
        test_pred = model(test_encoded, bos_input)
        model.train()
    decode = list(dataset.output_sequnces_vocab.keys())
    out_str = ''
    for i in test_pred.squeeze().cpu().detach().tolist():
        out_str += decode[i]
    print(out_str)

    torch.save(model.state_dict(), f'./seq2seq_chkpt_{epoch}.pth')

epoch: 0, step: 0, loss: 2.629819631576538
epoch: 0, step: 10, loss: 1.638836145401001
epoch: 0, step: 20, loss: 1.301824688911438
epoch: 0, step: 30, loss: 1.0219420194625854
epoch: 0, step: 40, loss: 0.8506795763969421
88-05-3#
epoch: 1, step: 0, loss: 0.6761842370033264
epoch: 1, step: 10, loss: 0.5632002949714661
epoch: 1, step: 20, loss: 0.465453565120697
epoch: 1, step: 30, loss: 0.4317287504673004
epoch: 1, step: 40, loss: 0.43588560819625854
888-55-44#
epoch: 2, step: 0, loss: 0.39425450563430786
epoch: 2, step: 10, loss: 0.3523031771183014
epoch: 2, step: 20, loss: 0.2975306510925293
epoch: 2, step: 30, loss: 0.29724907875061035
epoch: 2, step: 40, loss: 0.3182416260242462
888-55-44#
epoch: 3, step: 0, loss: 0.33475661277770996
epoch: 3, step: 10, loss: 0.3043808937072754
epoch: 3, step: 20, loss: 0.2866644859313965
epoch: 3, step: 30, loss: 0.263384073972702
epoch: 3, step: 40, loss: 0.2779485881328583
888-55-44#
epoch: 4, step: 0, loss: 0.23140473663806915
epoch: 4, step: 10

KeyboardInterrupt: ignored

In [18]:
test_pred

tensor([[2]])